<a href="https://colab.research.google.com/github/alebjanes/fire-susceptibility-mapping/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import load
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation, Input, Concatenate, Conv2DTranspose
from tensorflow.keras import regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
%load_ext tensorboard
from tensorboard.plugins.hparams import api as hp
import logging
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Data


In [ ]:
patch_size = 25
mid_pixel = int(patch_size / 2 - 0.5)

Dataset = np.load('/content/drive/My Drive/MT/Samples/samples' + str(patch_size)+ 'x' + str(patch_size) + '_v3.npy')

In [ ]:
X = Dataset[:,:,:,1:21]
target = Dataset[:,mid_pixel,mid_pixel,0]
y = np.expand_dims(target, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X=X_train
y=y_train

del y_test
del X_test
del target
del X_train
del y_train

# Pre processing

In [ ]:
#Elevacion capa 9 - Separacion de datos en dos grupos segun elevacion

#Set values:
threshold = 300
minval = -0.96570312976837
maxval = 4729.0986328125

n = 0
j = 0
threshold_norm = (threshold - minval) / (maxval - minval)

for i in range(Dataset.shape[0]):
  patch = np.expand_dims(Dataset[i, :, :, :], axis=0)
  elev = Dataset[i, mid_pixel, mid_pixel, 10]

  if elev < threshold_norm:
    n += 1
    if n == 1:
      X_class1 = patch
      print('X_class1:', X_class1.shape)
    else:
      X_class1 = np.concatenate((X_class1, patch), axis=0)

  else:
    j += 1
    if j == 1:
      X_class2 = patch
      print('X_class2:', X_class2.shape)
    else:
      X_class2 = np.concatenate((X_class2, patch), axis=0)

print("X class 1 (< threshold):", X_class1.shape)
print("X class 2 (>= threshold):", X_class2.shape)


X_class2: (1, 25, 25, 21)
X_class1: (1, 25, 25, 21)
X class 1 (< threshold): (20305, 25, 25, 21)
X class 2 (>= threshold): (9839, 25, 25, 21)


In [ ]:
#Separacion de datos en dos grupos segun clase
i = 0
n = 0
j = 0

for i in range(Dataset.shape[0]):
  patch = np.expand_dims(Dataset[i, :, :, :], axis=0)
  clase = np.where(Dataset[i, mid_pixel, mid_pixel, 15:] == 1)
  if clase[0].size == 0: clase = 5
  else: clase = int(clase[0])

  if clase == 0:
    n += 1
    if n == 1:
      X_class1 = patch
      print('X_class1:', X_class1.shape)
    else:
      X_class1 = np.concatenate((X_class1, patch), axis=0)

  else:
    j += 1
    if j == 1:
      X_class2 = patch
      print('X_class2:', X_class2.shape)
    else:
      X_class2 = np.concatenate((X_class2, patch), axis=0)

print("X class 1 (< threshold):", X_class1.shape)
print("X class 2 (>= threshold):", X_class2.shape)

X_class1: (1, 25, 25, 21)
X_class2: (1, 25, 25, 21)
X class 1 (< threshold): (15596, 25, 25, 21)
X class 2 (>= threshold): (14548, 25, 25, 21)


In [ ]:
np.save('/content/drive/My Drive/paper/samples/X_25_LULC_class1.npy', X_class1)
np.save('/content/drive/My Drive/paper/samples/X_25_LULC_class2.npy', X_class2)

In [ ]:
X_class1 = np.load('/content/drive/My Drive/paper/samples/X_25_LULC_class1.npy')
X_class2 = np.load('/content/drive/My Drive/paper/samples/X_25_LULC_class2.npy')

In [ ]:
#TRAIN/TEST SPLIT

X1 = X_class1[:, :, :, 1:21]
target1 = X_class1[:, mid_pixel, mid_pixel, 0]
y1 = np.expand_dims(target1, axis=1)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)


X2 = X_class2[:, :, :, 1:21]
target2 = X_class2[:, mid_pixel, mid_pixel, 0]
y2 = np.expand_dims(target2, axis=1)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [ ]:
X_train2.shape

(11638, 25, 25, 20)

In [ ]:
print('Clase verde -----------------------\n')

print('Total muestras:', X_class1.shape[0])
print('Total muestras de incendios:', np.sum(y_train1) + np.sum(y_test1))
print('Total de muestras de no incendio:', X_class1.shape[0] - (np.sum(y_train1) + np.sum(y_test1)), '\n' )

print('Set de validacion \n')

print('ones:', np.sum(y_test1))
print('zeros:', X_test1.shape[0] - np.sum(y_test1))

print('\n')
print('Otras clases -----------------------\n')

print('Total muestras:', X_class2.shape[0])
print('Total muestras de incendios:', np.sum(y_train2) + np.sum(y_test2))
print('Total de muestras de no incendio:', X_class2.shape[0] - (np.sum(y_train2) + np.sum(y_test2)), '\n' )

print('Set de validacion \n')

print('ones:', np.sum(y_test2))
print('zeros:', X_test2.shape[0] - np.sum(y_test2))

Clase verde -----------------------

Total muestras: 15596
Total muestras de incendios: 6892.0
Total de muestras de no incendio: 8704.0 

Set de validacion 

ones: 1397.0
zeros: 1723.0


Otras clases -----------------------

Total muestras: 14548
Total muestras de incendios: 8180.0
Total de muestras de no incendio: 6368.0 

Set de validacion 

ones: 1653.0
zeros: 1257.0


# Models

In [ ]:
CNN1 = tf.keras.models.load_model('/content/drive/My Drive/MT/checkpoints/CNN1/CNN1_25')

y_pred_CNN1 = CNN1.predict(X_test).ravel()

y_pred_CNN1_classes = np.where(y_pred_CNN1 > 0.5, 1, 0)

from sklearn.metrics import accuracy_score

CNN1.evaluate(X_test, y_test)

In [ ]:
#MODELS

def cnn1(input_shape, batch_normalization, momentum, activation, loss, lr, optimizer):
    model = Sequential()
    model.add(Conv2D(32, 3, strides = 1, padding = 'same', input_shape = input_shape))
    model.add(Activation(activation = activation))
    if batch_normalization: 
        model.add(BatchNormalization(momentum=momentum, scale = False, renorm = True))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(64, 3, strides = 1, padding = 'same'))
    model.add(Activation(activation = activation))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(128, 3, strides = 1, padding = 'same'))
    model.add(Activation(activation = activation))
    model.add(Flatten())
    model.add(Dense(128, activation = activation))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = activation))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation  = activation))
    model.add(Dropout(0.5))
    if loss == 'binary_crossentropy':
        model.add(Dense(1, activation = 'sigmoid'))
    elif loss == 'sparse_categorical_crossentropy':
        model.add(Dense(2, activation = 'softmax'))  

    if optimizer == 'adam':
      model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.9999, epsilon=1e-07), metrics=['accuracy' 
                                                                                                                                        #, tf.keras.metrics.AUC()
                                                                                                                                        ])
    elif optimizer == 'sgd':
      model.compile(loss=loss, optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9), metrics=['accuracy'])   

    #model.summary()

    return model

def inputlayers(input_shape, batch_normalization, momentum, activation):
  input = Input(shape=input_shape)
  x = Conv2D(32, kernel_size=(3, 3), activation=activation, padding = 'same')(input)
  if batch_normalization: 
    x = BatchNormalization(momentum=0.99)(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Conv2D(64, kernel_size=(3, 3), activation=activation, padding = 'same')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  conv = Conv2D(128, kernel_size=(3, 3), activation=activation, padding = 'same')(x)

  return input, conv

def multi_input(patch_size, batch_normalization, momentum, activation, loss, lr):
  #Climatic features
  input1, feature1 = inputlayers((patch_size, patch_size, 6), batch_normalization, momentum = momentum, activation = activation)

  #NDVI feature
  input2, feature2 = inputlayers((patch_size, patch_size, 1), batch_normalization, momentum = momentum, activation = activation)

  #Topograhic features
  input3, feature3 = inputlayers((patch_size, patch_size, 4), batch_normalization, momentum = momentum, activation = activation)

  #Human-related features
  input4, feature4 = inputlayers((patch_size, patch_size, 3), batch_normalization, momentum = momentum , activation = activation)

  #Human-related features
  input5, feature5 = inputlayers((patch_size, patch_size, 6), batch_normalization, momentum = momentum , activation = activation)

  merge = Concatenate(axis = -1)([feature1, feature2, feature3, feature4, feature5])
  
  x = Conv2DTranspose(32, kernel_size=(3,3), dilation_rate=2)(merge)
  x = Conv2D(32, kernel_size=(3, 3), activation=activation, padding = 'same')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Conv2D(64, kernel_size=(3, 3), activation=activation, padding = 'same')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  conv = Conv2D(128, kernel_size=(3, 3), activation=activation, padding = 'same')(x)

  flatten = Flatten()(conv)

  FC2 = Dense(128, activation = activation, kernel_regularizer=regularizers.l1_l2(l1 = 0.001, l2 = 0.01))(flatten)
  dropout2 = Dropout(0.5)(FC2)
  FC3 = Dense(64, activation = activation, kernel_regularizer=regularizers.l1_l2(l1 = 0.001, l2 = 0.01))(dropout2)
  dropout3 = Dropout(0.5)(FC3)
  FC4 = Dense(32, activation = activation, kernel_regularizer=regularizers.l1_l2(l1 = 0.001, l2 = 0.01))(dropout3)
  dropout4 = Dropout(0.5)(FC4)
  if loss == 'binary_crossentropy':
    output = Dense(1, activation = 'sigmoid')(dropout4)
  elif loss == 'sparse_categorical_crossentropy':
    output = Dense(2, activation = 'softmax')(dropout4)  
  
  model = Model(inputs=[input1, input2, input3, input4, input5], outputs = output)
  
  model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.9999, epsilon=1e-07), metrics=['accuracy'])

#kernel_regularizer=regularizers.l1_l2(l1 = 0.001,l2 = 0.01)

  return model

In [ ]:
#Callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=15, min_lr=0.0001, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=200, min_delta=0.01, restore_best_weights=True)

In [ ]:
#CNN! XCLASS 1
model1 = cnn1(input_shape = (25, 25, 20), batch_normalization=True, momentum = 0.99, activation = 'relu', loss='binary_crossentropy', lr = 0.0001, optimizer = 'adam')

history1 = model1.fit(X_train1, y_train1, batch_size = 32, epochs = 500, callbacks = [reduce_lr, early_stop], validation_data = (X_test1, y_test1), verbose = 1)

In [ ]:
model2 = cnn1(input_shape = (25, 25, 20), batch_normalization=True, momentum = 0.99, activation = 'relu', loss='binary_crossentropy', lr = 0.0001, optimizer = 'adam')

history2 = model2.fit(X_train2, y_train2, batch_size = 32, epochs = 300, callbacks = [reduce_lr, early_stop], validation_data = (X_test2, y_test2), verbose = 1)

In [ ]:
#def data_separator(batch_size, X, y):
#  while True:

#    climatic = X[:,:,:,0:6]
#    vegetation = X[:,:,:,6:7]
#    topographic = X[:,:,:,7:11]
#    human_related = X[:,:,:,11:14]
#    LULC = X[:,:,:,14:21]
    
#    yield

In [ ]:
climatic1 = X_train1[:,:,:,0:6]
vegetation1 = X_train1[:,:,:,6:7]
topographic1 = X_train1[:,:,:,7:11]
human_related1 = X_train1[:,:,:,11:14]
LULC1 = X_train1[:,:,:,14:21]

climatic_val1 = X_test1[:,:,:,0:6]
vegetation_val1 = X_test1[:,:,:,6:7]
topographic_val1 = X_test1[:,:,:,7:11]
human_related_val1 = X_test1[:,:,:,11:14]
LULC_val1 = X_test1[:,:,:,14:21]

In [ ]:
MI_1 = multi_input(patch_size = 25, batch_normalization = True, momentum = 0.99, activation = 'relu', loss = 'binary_crossentropy', lr = 0.0001)

history = MI_1.fit([climatic1, vegetation1, topographic1, human_related1, LULC1], y_train1,
              batch_size=32,
              epochs=5000,
              callbacks=[reduce_lr],
              validation_data = ([climatic_val1, vegetation_val1, topographic_val1, human_related_val1, LULC_val1], y_test1), verbose = 1)

In [ ]:
#modelcnn.save('/content/drive/My Drive/MT/checkpoints/CNN1/CNN1_')

In [ ]:
MI_1.save('/content/drive/My Drive/MT/paper/models/MI_1_LULC')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/MT/paper/models/MI_1_LULC/assets


In [ ]:
climatic2 = X_train2[:,:,:,0:6]
vegetation2 = X_train2[:,:,:,6:7]
topographic2 = X_train2[:,:,:,7:11]
human_related2 = X_train2[:,:,:,11:14]
LULC2 = X_train2[:,:,:,14:21]

climatic_val2 = X_test2[:,:,:,0:6]
vegetation_val2 = X_test2[:,:,:,6:7]
topographic_val2 = X_test2[:,:,:,7:11]
human_related_val2 = X_test2[:,:,:,11:14]
LULC_val2 = X_test2[:,:,:,14:21]

In [ ]:
MI_2 = multi_input(patch_size = 25, batch_normalization = True, momentum = 0.99, activation = 'relu', loss = 'binary_crossentropy', lr = 0.0001)

history2 = MI_2.fit([climatic2, vegetation2, topographic2, human_related2, LULC2], y_train2,
              batch_size=32,
              epochs=300,
              callbacks=[reduce_lr, early_stop],
              validation_data = ([climatic_val2, vegetation_val2, topographic_val2, human_related_val2, LULC_val2], y_test2), verbose = 1)

# CV

In [ ]:
X.shape

(28084, 5, 5, 20)

In [ ]:
#CV
num_folds = 5

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=False, random_state = 42)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
fold_no = 1
for train, test in kfold.split(X, y):
  
  n = 0
  j = 0 

  for i in range(X[train].shape[0]):
    patch = np.expand_dims(X[train][i, :, :, :], axis=0)

    if X[train][i, mid_pixel, mid_pixel, 14] == 1:
      n += 1
      if n == 1:
        X_train1 = patch
        y_train1 = y[train][i]
      else: 
        X_train1 = np.concatenate((X_train1, patch), axis = 0)
        y_train1 = np.concatenate((y_train1, y[train][i]), axis = 0)

    else:
      j += 1
      if j == 1:
        X_train2 = patch
        y_train2 = y[train][i]
      else:
        X_train2 = np.concatenate((X_train2, patch), axis=0)
        y_train2 = np.concatenate((y_train2, y[train][i]), axis = 0)

    if i == 100 or i == 500 or i == 1000 or i == 2000 or i == 10000 or i == 15000:    
      print('Clase 0:', n, "//// Otras clases:", j)


  np.save('/content/drive/My Drive/paper/samples/kfold/X_train1_fold' + str(fold_no) + '_7.npy', X_train1)
  np.save('/content/drive/My Drive/paper/samples/kfold/X_train2_fold' + str(fold_no) + '_7.npy', X_train2)
  np.save('/content/drive/My Drive/paper/samples/kfold/y_train1_fold' + str(fold_no) + '_7.npy', y_train1)
  np.save('/content/drive/My Drive/paper/samples/kfold/y_train2_fold' + str(fold_no) + '_7.npy', y_train2)
  
  # Increase fold number
  fold_no = fold_no + 1
  print(fold_no)

Clase 0: 58 //// Otras clases: 43
Clase 0: 259 //// Otras clases: 242
Clase 0: 516 //// Otras clases: 485
Clase 0: 995 //// Otras clases: 1006
Clase 0: 5072 //// Otras clases: 4929
Clase 0: 7665 //// Otras clases: 7336
2
Clase 0: 54 //// Otras clases: 47
Clase 0: 265 //// Otras clases: 236
Clase 0: 521 //// Otras clases: 480
Clase 0: 1032 //// Otras clases: 969
Clase 0: 5136 //// Otras clases: 4865
Clase 0: 7729 //// Otras clases: 7272
3
Clase 0: 54 //// Otras clases: 47
Clase 0: 265 //// Otras clases: 236
Clase 0: 521 //// Otras clases: 480
Clase 0: 1032 //// Otras clases: 969
Clase 0: 5098 //// Otras clases: 4903
Clase 0: 7669 //// Otras clases: 7332
4
Clase 0: 54 //// Otras clases: 47
Clase 0: 265 //// Otras clases: 236
Clase 0: 521 //// Otras clases: 480
Clase 0: 1032 //// Otras clases: 969
Clase 0: 5098 //// Otras clases: 4903
Clase 0: 7649 //// Otras clases: 7352
5
Clase 0: 54 //// Otras clases: 47
Clase 0: 265 //// Otras clases: 236
Clase 0: 521 //// Otras clases: 480
Clase 0: 1

In [ ]:
fold_no = 1
for train, test in kfold.split(X, y):
  
  n = 0
  j = 0 

  for i in range(X[test].shape[0]):
    patch = np.expand_dims(X[test][i, :, :, :], axis=0)

    if X[test][i, mid_pixel, mid_pixel, 14] == 1:
      n += 1
      if n == 1:
        X_test1 = patch
        y_test1 = y[test][i]
      else: 
        X_test1 = np.concatenate((X_test1, patch), axis = 0)
        y_test1 = np.concatenate((y_test1, y[test][i]), axis = 0)

    else:
      j += 1
      if j == 1:
        X_test2 = patch
        y_test2 = y[test][i]
      else:
        X_test2 = np.concatenate((X_test2, patch), axis=0)
        y_test2 = np.concatenate((y_test2, y[test][i]), axis = 0)

    if i == 100 or i == 500 or i == 1000 or i == 2000 or i == 10000 or i == 15000:    
      print('Clase 0:', n, "//// Otras clases:", j)


  np.save('/content/drive/My Drive/paper/samples/kfold/X_test1_fold' + str(fold_no) + '_7.npy', X_test1)
  np.save('/content/drive/My Drive/paper/samples/kfold/X_test2_fold' + str(fold_no) + '_7.npy', X_test2)
  np.save('/content/drive/My Drive/paper/samples/kfold/y_test1_fold' + str(fold_no) + '_7.npy', y_test1)
  np.save('/content/drive/My Drive/paper/samples/kfold/y_test2_fold' + str(fold_no) + '_7.npy', y_test2)
  
  # Increase fold number
  fold_no = fold_no + 1
  print(fold_no)

Clase 0: 54 //// Otras clases: 47
Clase 0: 265 //// Otras clases: 236
Clase 0: 521 //// Otras clases: 480
Clase 0: 1032 //// Otras clases: 969
2
Clase 0: 58 //// Otras clases: 43
Clase 0: 259 //// Otras clases: 242
Clase 0: 516 //// Otras clases: 485
Clase 0: 995 //// Otras clases: 1006
3
Clase 0: 51 //// Otras clases: 50
Clase 0: 267 //// Otras clases: 234
Clase 0: 509 //// Otras clases: 492
Clase 0: 1015 //// Otras clases: 986
4
Clase 0: 55 //// Otras clases: 46
Clase 0: 260 //// Otras clases: 241
Clase 0: 530 //// Otras clases: 471
Clase 0: 1034 //// Otras clases: 967
5
Clase 0: 63 //// Otras clases: 38
Clase 0: 268 //// Otras clases: 233
Clase 0: 549 //// Otras clases: 452
Clase 0: 1073 //// Otras clases: 928
6


In [ ]:
acc_per_fold_1 = []
loss_per_fold_1 = []

acc_per_fold_train_1 = []
loss_per_fold_train_1 = []

# K-fold Cross Validation model evaluation MULTI INPUT
fold_no = 1
for fold_no in range(1, 6):

  #LOAD FOLD DATASET
  X_test1 = np.load('/content/drive/My Drive/paper/samples/kfold/X_test1_fold' + str(fold_no) + '_7.npy')
  y_test1 = np.load('/content/drive/My Drive/paper/samples/kfold/y_test1_fold' + str(fold_no) + '_7.npy')

  X_train1 = np.load('/content/drive/My Drive/paper/samples/kfold/X_train1_fold' + str(fold_no) + '_7.npy')
  y_train1 = np.load('/content/drive/My Drive/paper/samples/kfold/y_train1_fold' + str(fold_no) + '_7.npy')

  #Multi input partition

  climatic1 = X_train1[:,:,:,0:6]
  vegetation1 = X_train1[:,:,:,6:7]
  topographic1 = X_train1[:,:,:,7:11]
  human_related1 = X_train1[:,:,:,11:14]
  LULC1 = X_train1[:,:,:,14:21]

  climatic_val1 = X_test1[:,:,:,0:6]
  vegetation_val1 = X_test1[:,:,:,6:7]
  topographic_val1 = X_test1[:,:,:,7:11]
  human_related_val1 = X_test1[:,:,:,11:14]
  LULC_val1 = X_test1[:,:,:,14:21]

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} / Multi input / GROUP 1...')

  MI = multi_input(patch_size = 7, batch_normalization = True, momentum = 0.99, activation = 'relu', loss = 'binary_crossentropy', lr = 0.0001)

  history_MI1 = MI.fit([climatic1, vegetation1, topographic1, human_related1, LULC1], y_train1,
              batch_size=32,
              epochs=5000,
              callbacks=[reduce_lr, early_stop],
              validation_data = ([climatic_val1, vegetation_val1, topographic_val1, human_related_val1, LULC_val1], y_test1), verbose = 0)
  
  # Generate generalization metrics on VALIDATION
  scores = MI.evaluate([climatic_val1, vegetation_val1, topographic_val1, human_related_val1, LULC_val1], y_test1, verbose=0)
  print(f'Score for fold {fold_no} on TESTING: {MI.metrics_names[0]} of {scores[0]}; {MI.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold_1.append(scores[1] * 100)
  loss_per_fold_1.append(scores[0])

  # Generate generalization metrics on TRAINING
  scores_train = MI.evaluate([climatic1, vegetation1, topographic1, human_related1, LULC1], y_train1, verbose=0)
  print(f'Score for fold {fold_no} on TRAINING: {MI.metrics_names[0]} of {scores_train[0]}; {MI.metrics_names[1]} of {scores_train[1]*100}%')
  acc_per_fold_train_1.append(scores_train[1] * 100)
  loss_per_fold_train_1.append(scores_train[0])

  # Increase fold number
  fold_no = fold_no + 1


  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold on validation set')
for i in range(0, len(acc_per_fold_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_1[i]} - Accuracy: {acc_per_fold_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_1)} (+- {np.std(acc_per_fold_1)})')
print(f'> Loss: {np.mean(loss_per_fold_1)}')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('Score per fold on training set')
for i in range(0, len(acc_per_fold_train_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_train_1[i]} - Accuracy: {acc_per_fold_train_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_train_1)} (+- {np.std(acc_per_fold_train_1)})')
print(f'> Loss: {np.mean(loss_per_fold_train_1)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 / Multi input / GROUP 1...
Restoring model weights from the end of the best epoch.
Epoch 00359: early stopping
Score for fold 1 on TESTING: loss of 1.5248509645462036; accuracy of 67.86355376243591%
Score for fold 1 on TRAINING: loss of 0.24959230422973633; accuracy of 89.21753168106079%
------------------------------------------------------------------------
Training for fold 2 / Multi input / GROUP 1...
Restoring model weights from the end of the best epoch.
Epoch 00210: early stopping
Score for fold 2 on TESTING: loss of 0.6608866453170776; accuracy of 66.29915237426758%
Score for fold 2 on TRAINING: loss of 0.6282060742378235; accuracy of 70.7156240940094%
------------------------------------------------------------------------
Training for fold 3 / Multi input / GROUP 1...
Restoring model weights from the end of the best epoch.
Epoch 00208: early stopping
Score for fold 3 on TESTING: loss 

In [ ]:
acc_per_fold_2 = []
loss_per_fold_2 = []

acc_per_fold_train_2 = []
loss_per_fold_train_2 = []

# K-fold Cross Validation model evaluation MULTI INPUT
fold_no = 1
for fold_no in range(1, 6):

  #LOAD FOLD DATASET
  X_test2 = np.load('/content/drive/My Drive/paper/samples/kfold/X_test2_fold' + str(fold_no) + '_7.npy')
  y_test2 = np.load('/content/drive/My Drive/paper/samples/kfold/y_test2_fold' + str(fold_no) + '_7.npy')

  X_train2 = np.load('/content/drive/My Drive/paper/samples/kfold/X_train2_fold' + str(fold_no) + '_7.npy')
  y_train2 = np.load('/content/drive/My Drive/paper/samples/kfold/y_train2_fold' + str(fold_no) + '_7.npy')

  #Multi input partition

  climatic2 = X_train2[:,:,:,0:6]
  vegetation2 = X_train2[:,:,:,6:7]
  topographic2 = X_train2[:,:,:,7:11]
  human_related2 = X_train2[:,:,:,11:14]
  LULC2 = X_train2[:,:,:,14:21]

  climatic_val2 = X_test2[:,:,:,0:6]
  vegetation_val2 = X_test2[:,:,:,6:7]
  topographic_val2 = X_test2[:,:,:,7:11]
  human_related_val2 = X_test2[:,:,:,11:14]
  LULC_val2 = X_test2[:,:,:,14:21]

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} / Multi input / GROUP 2...')

  MI2 = multi_input(patch_size = 7, batch_normalization = True, momentum = 0.99, activation = 'relu', loss = 'binary_crossentropy', lr = 0.0001)

  history_MI2 = MI2.fit([climatic2, vegetation2, topographic2, human_related2, LULC2], y_train2,
              batch_size=32,
              epochs=300,
              callbacks=[reduce_lr, early_stop],
              validation_data = ([climatic_val2, vegetation_val2, topographic_val2, human_related_val2, LULC_val2], y_test2), verbose = 0)
  
  # Generate generalization metrics on VALIDATION
  scores = MI2.evaluate([climatic_val2, vegetation_val2, topographic_val2, human_related_val2, LULC_val2], y_test2, verbose=0)
  print(f'Score for fold {fold_no} on TESTING: {MI2.metrics_names[0]} of {scores[0]}; {MI2.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold_2.append(scores[1] * 100)
  loss_per_fold_2.append(scores[0])

  # Generate generalization metrics on TRAINING
  scores_train = MI2.evaluate([climatic2, vegetation2, topographic2, human_related2, LULC2], y_train2, verbose=0)
  print(f'Score for fold {fold_no} on TRAINING: {MI2.metrics_names[0]} of {scores_train[0]}; {MI2.metrics_names[1]} of {scores_train[1]*100}%')
  acc_per_fold_train_2.append(scores_train[1] * 100)
  loss_per_fold_train_2.append(scores_train[0])

  # Increase fold number
  fold_no = fold_no + 1


  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold on validation set')
for i in range(0, len(acc_per_fold_2)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_2[i]} - Accuracy: {acc_per_fold_2[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_2)} (+- {np.std(acc_per_fold_2)})')
print(f'> Loss: {np.mean(loss_per_fold_2)}')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('Score per fold on training set')
for i in range(0, len(acc_per_fold_train_2)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_train_2[i]} - Accuracy: {acc_per_fold_train_2[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_train_2)} (+- {np.std(acc_per_fold_train_2)})')
print(f'> Loss: {np.mean(loss_per_fold_train_2)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 / Multi input / GROUP 2...
Restoring model weights from the end of the best epoch.
Epoch 00206: early stopping
Score for fold 1 on TESTING: loss of 0.8162353038787842; accuracy of 60.9422504901886%
Score for fold 1 on TRAINING: loss of 0.7902636528015137; accuracy of 64.27212357521057%
------------------------------------------------------------------------
Training for fold 2 / Multi input / GROUP 2...
Restoring model weights from the end of the best epoch.
Epoch 00207: early stopping
Score for fold 2 on TESTING: loss of 0.7323052287101746; accuracy of 61.83234453201294%
Score for fold 2 on TRAINING: loss of 0.7181349396705627; accuracy of 63.63201141357422%
------------------------------------------------------------------------
Training for fold 3 / Multi input / GROUP 2...
Restoring model weights from the end of the best epoch.
Epoch 00207: early stopping
Score for fold 3 on TESTING: loss o

In [ ]:
acc_per_fold_1 = []
loss_per_fold_1 = []

acc_per_fold_train_1 = []
loss_per_fold_train_1 = []

# K-fold Cross Validation model evaluation CNN1
fold_no = 1
for fold_no in range(1, 6):

  #LOAD FOLD DATASET
  X_test1 = np.load('/content/drive/My Drive/paper/samples/kfold/X_test1_fold' + str(fold_no) + '_7.npy')
  y_test1 = np.load('/content/drive/My Drive/paper/samples/kfold/y_test1_fold' + str(fold_no) + '_7.npy')

  X_train1 = np.load('/content/drive/My Drive/paper/samples/kfold/X_train1_fold' + str(fold_no) + '_7.npy')
  y_train1 = np.load('/content/drive/My Drive/paper/samples/kfold/y_train1_fold' + str(fold_no) + '_7.npy')

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} / CNN1 / GROUP 1...')

  CNN1 = cnn1(input_shape = (7, 7, 20), batch_normalization=True, momentum = 0.99, activation = 'relu', loss='binary_crossentropy', lr = 0.0001, optimizer = 'adam')

  history_CNN1 = CNN1.fit(X_train1, y_train1, batch_size = 32, epochs = 500, callbacks = [reduce_lr, early_stop], validation_data = (X_test1, y_test1), verbose = 0)
  
  # Generate generalization metrics on VALIDATION
  scores = CNN1.evaluate(X_test1, y_test1, verbose=0)
  print(f'Score for fold {fold_no} on TESTING: {CNN1.metrics_names[0]} of {scores[0]}; {CNN1.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold_1.append(scores[1] * 100)
  loss_per_fold_1.append(scores[0])

  # Generate generalization metrics on TRAINING
  scores_train = CNN1.evaluate(X_train1, y_train1, verbose=0)
  print(f'Score for fold {fold_no} on TRAINING: {CNN1.metrics_names[0]} of {scores_train[0]}; {CNN1.metrics_names[1]} of {scores_train[1]*100}%')
  acc_per_fold_train_1.append(scores_train[1] * 100)
  loss_per_fold_train_1.append(scores_train[0])

  # Increase fold number
  fold_no = fold_no + 1


  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold on validation set')
for i in range(0, len(acc_per_fold_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_1[i]} - Accuracy: {acc_per_fold_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_1)} (+- {np.std(acc_per_fold_1)})')
print(f'> Loss: {np.mean(loss_per_fold_1)}')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('Score per fold on training set')
for i in range(0, len(acc_per_fold_train_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_train_1[i]} - Accuracy: {acc_per_fold_train_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_train_1)} (+- {np.std(acc_per_fold_train_1)})')
print(f'> Loss: {np.mean(loss_per_fold_train_1)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 / CNN1 / GROUP 1...
Restoring model weights from the end of the best epoch.
Epoch 00307: early stopping
Score for fold 1 on TESTING: loss of 0.6495362520217896; accuracy of 66.35547280311584%
Score for fold 1 on TRAINING: loss of 0.6238362789154053; accuracy of 66.34399890899658%
------------------------------------------------------------------------
Training for fold 2 / CNN1 / GROUP 1...
Restoring model weights from the end of the best epoch.
Epoch 00390: early stopping
Score for fold 2 on TESTING: loss of 0.6331994533538818; accuracy of 66.55641198158264%
Score for fold 2 on TRAINING: loss of 0.6014804840087891; accuracy of 67.80144572257996%
------------------------------------------------------------------------
Training for fold 3 / CNN1 / GROUP 1...
Restoring model weights from the end of the best epoch.
Epoch 00316: early stopping
Score for fold 3 on TESTING: loss of 0.6369702816009521

In [ ]:
acc_per_fold_1 = []
loss_per_fold_1 = []

acc_per_fold_train_1 = []
loss_per_fold_train_1 = []

# K-fold Cross Validation model evaluation CNN1
fold_no = 1
for fold_no in range(1, 6):

  #LOAD FOLD DATASET
  X_test2 = np.load('/content/drive/My Drive/paper/samples/kfold/X_test2_fold' + str(fold_no) + '_7.npy')
  y_test2 = np.load('/content/drive/My Drive/paper/samples/kfold/y_test2_fold' + str(fold_no) + '_7.npy')

  X_train2 = np.load('/content/drive/My Drive/paper/samples/kfold/X_train2_fold' + str(fold_no) + '_7.npy')
  y_train2 = np.load('/content/drive/My Drive/paper/samples/kfold/y_train2_fold' + str(fold_no) + '_7.npy')

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} / CNN1 / GROUP 2...')

  CNN1 = cnn1(input_shape = (7, 7, 20), batch_normalization=True, momentum = 0.99, activation = 'relu', loss='binary_crossentropy', lr = 0.0001, optimizer = 'adam')

  history_CNN1 = CNN1.fit(X_train2, y_train2, batch_size = 32, epochs = 500, callbacks = [reduce_lr, early_stop], validation_data = (X_test1, y_test1), verbose = 0)
  
  # Generate generalization metrics on VALIDATION
  scores = CNN1.evaluate(X_test2, y_test2, verbose=0)
  print(f'Score for fold {fold_no} on TESTING: {CNN1.metrics_names[0]} of {scores[0]}; {CNN1.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold_1.append(scores[1] * 100)
  loss_per_fold_1.append(scores[0])

  # Generate generalization metrics on TRAINING
  scores_train = CNN1.evaluate(X_train2, y_train2, verbose=0)
  print(f'Score for fold {fold_no} on TRAINING: {CNN1.metrics_names[0]} of {scores_train[0]}; {CNN1.metrics_names[1]} of {scores_train[1]*100}%')
  acc_per_fold_train_1.append(scores_train[1] * 100)
  loss_per_fold_train_1.append(scores_train[0])

  # Increase fold number
  fold_no = fold_no + 1


  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold on validation set')
for i in range(0, len(acc_per_fold_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_1[i]} - Accuracy: {acc_per_fold_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_1)} (+- {np.std(acc_per_fold_1)})')
print(f'> Loss: {np.mean(loss_per_fold_1)}')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('Score per fold on training set')
for i in range(0, len(acc_per_fold_train_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_train_1[i]} - Accuracy: {acc_per_fold_train_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_train_1)} (+- {np.std(acc_per_fold_train_1)})')
print(f'> Loss: {np.mean(loss_per_fold_train_1)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 / CNN1 / GROUP 2...
Restoring model weights from the end of the best epoch.
Epoch 00319: early stopping
Score for fold 1 on TESTING: loss of 0.6539201736450195; accuracy of 59.15653705596924%
Score for fold 1 on TRAINING: loss of 0.6394328474998474; accuracy of 60.742151737213135%
------------------------------------------------------------------------
Training for fold 2 / CNN1 / GROUP 2...
Restoring model weights from the end of the best epoch.
Epoch 00253: early stopping
Score for fold 2 on TESTING: loss of 0.668300211429596; accuracy of 59.013354778289795%
Score for fold 2 on TRAINING: loss of 0.6570395231246948; accuracy of 59.96553897857666%
------------------------------------------------------------------------
Training for fold 3 / CNN1 / GROUP 2...
Restoring model weights from the end of the best epoch.
Epoch 00270: early stopping
Score for fold 3 on TESTING: loss of 0.643504083156585

In [ ]:
acc_per_fold_1 = []
loss_per_fold_1 = []

acc_per_fold_train_1 = []
loss_per_fold_train_1 = []

CNN1 = tf.keras.models.load_model('/content/drive/My Drive/MT/checkpoints/CNN1/CNN1_25')

# K-fold Cross Validation model evaluation CNN1
fold_no = 1
for fold_no in range(1, 6):

  #LOAD FOLD DATASET
  X_test2 = np.load('/content/drive/My Drive/paper/samples/kfold/X_test2_fold' + str(fold_no) + '.npy')
  y_test2 = np.load('/content/drive/My Drive/paper/samples/kfold/y_test2_fold' + str(fold_no) + '.npy')

  X_train2 = np.load('/content/drive/My Drive/paper/samples/kfold/X_train2_fold' + str(fold_no) + '.npy')
  y_train2 = np.load('/content/drive/My Drive/paper/samples/kfold/y_train2_fold' + str(fold_no) + '.npy')

  X_test1 = np.load('/content/drive/My Drive/paper/samples/kfold/X_test1_fold' + str(fold_no) + '.npy')
  y_test1 = np.load('/content/drive/My Drive/paper/samples/kfold/y_test1_fold' + str(fold_no) + '.npy')

  X_train1 = np.load('/content/drive/My Drive/paper/samples/kfold/X_train1_fold' + str(fold_no) + '.npy')
  y_train1 = np.load('/content/drive/My Drive/paper/samples/kfold/y_train1_fold' + str(fold_no) + '.npy')

  print('\n------------------------------------------------------------------------\n')
  print(f'Predictions for fold {fold_no} / with pretrained CNN1 / GROUP 1...')

  # Generate generalization metrics on VALIDATION
  scores = CNN1.evaluate(X_test1, y_test1, verbose=0)
  print(f'Score for fold {fold_no} on TESTING: {CNN1.metrics_names[0]} of {scores[0]}; {CNN1.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold_1.append(scores[1] * 100)
  loss_per_fold_1.append(scores[0])

  # Generate generalization metrics on TRAINING
  scores_train = CNN1.evaluate(X_train1, y_train1, verbose=0)
  print(f'Score for fold {fold_no} on TRAINING: {CNN1.metrics_names[0]} of {scores_train[0]}; {CNN1.metrics_names[1]} of {scores_train[1]*100}%')
  acc_per_fold_train_1.append(scores_train[1] * 100)
  loss_per_fold_train_1.append(scores_train[0])

  print('\n------------------------------------------------------------------------\n')
  print(f'Predictions for fold {fold_no} / with pretrained CNN1 / GROUP 2...')

  # Generate generalization metrics on VALIDATION
  scores = CNN1.evaluate(X_test2, y_test2, verbose=0)
  print(f'Score for fold {fold_no} on TESTING: {CNN1.metrics_names[0]} of {scores[0]}; {CNN1.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold_1.append(scores[1] * 100)
  loss_per_fold_1.append(scores[0])

  # Generate generalization metrics on TRAINING
  scores_train = CNN1.evaluate(X_train2, y_train2, verbose=0)
  print(f'Score for fold {fold_no} on TRAINING: {CNN1.metrics_names[0]} of {scores_train[0]}; {CNN1.metrics_names[1]} of {scores_train[1]*100}%')
  acc_per_fold_train_1.append(scores_train[1] * 100)
  loss_per_fold_train_1.append(scores_train[0])

  # Increase fold number
  fold_no = fold_no + 1


  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold on validation set')
for i in range(0, len(acc_per_fold_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_1[i]} - Accuracy: {acc_per_fold_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_1)} (+- {np.std(acc_per_fold_1)})')
print(f'> Loss: {np.mean(loss_per_fold_1)}')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('Score per fold on training set')
for i in range(0, len(acc_per_fold_train_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_train_1[i]} - Accuracy: {acc_per_fold_train_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_train_1)} (+- {np.std(acc_per_fold_train_1)})')
print(f'> Loss: {np.mean(loss_per_fold_train_1)}')
print('------------------------------------------------------------------------')


------------------------------------------------------------------------

Predictions for fold 1 / with pretrained CNN1 / GROUP 1...
Score for fold 1 on TESTING: loss of 0.17298440635204315; accuracy of 92.9421067237854%
Score for fold 1 on TRAINING: loss of 0.17646586894989014; accuracy of 93.12286376953125%

------------------------------------------------------------------------

Predictions for fold 1 / with pretrained CNN1 / GROUP 2...
Score for fold 1 on TESTING: loss of 0.2602132558822632; accuracy of 88.48326802253723%
Score for fold 1 on TRAINING: loss of 0.24327576160430908; accuracy of 89.82452154159546%

------------------------------------------------------------------------

Predictions for fold 2 / with pretrained CNN1 / GROUP 1...
Score for fold 2 on TESTING: loss of 0.1678781509399414; accuracy of 93.62295866012573%
Score for fold 2 on TRAINING: loss of 0.17774778604507446; accuracy of 92.951101064682%

-----------------------------------------------------------------

In [ ]:
acc_per_fold_1 = []
loss_per_fold_1 = []

acc_per_fold_train_1 = []
loss_per_fold_train_1 = []

CNN1 = tf.keras.models.load_model('/content/drive/My Drive/MT/checkpoints/CNN1/CNN1_25')

# K-fold Cross Validation model evaluation CNN1
fold_no = 1
for fold_no in range(1, 6):

  #LOAD FOLD DATASET
  X_test2 = np.load('/content/drive/My Drive/paper/samples/kfold/X_test2_fold' + str(fold_no) + '.npy')
  y_test2 = np.load('/content/drive/My Drive/paper/samples/kfold/y_test2_fold' + str(fold_no) + '.npy')

  X_train2 = np.load('/content/drive/My Drive/paper/samples/kfold/X_train2_fold' + str(fold_no) + '.npy')
  y_train2 = np.load('/content/drive/My Drive/paper/samples/kfold/y_train2_fold' + str(fold_no) + '.npy')

  X_test1 = np.load('/content/drive/My Drive/paper/samples/kfold/X_test1_fold' + str(fold_no) + '.npy')
  y_test1 = np.load('/content/drive/My Drive/paper/samples/kfold/y_test1_fold' + str(fold_no) + '.npy')

  X_train1 = np.load('/content/drive/My Drive/paper/samples/kfold/X_train1_fold' + str(fold_no) + '.npy')
  y_train1 = np.load('/content/drive/My Drive/paper/samples/kfold/y_train1_fold' + str(fold_no) + '.npy')

  X_train = np.concatenate((X_train1, X_train2), axis = 0)
  X_test = np.concatenate((X_test1, X_test2), axis = 0)
  y_train = np.concatenate((y_train1, y_train2), axis = 0)
  y_test = np.concatenate((y_test1, y_test2), axis = 0)

  print('\n------------------------------------------------------------------------\n')
  print('Training set size', X_train.shape[0])
  print('Testing set size', X_test.shape[0])
  print('\n------------------------------------------------------------------------\n')

  print('\n------------------------------------------------------------------------\n')
  print(f'Predictions for fold {fold_no} / with pretrained CNN1 / Complete dataset...')

  # Generate generalization metrics on VALIDATION
  scores = CNN1.evaluate(X_test, y_test, verbose=0)
  print(f'Score for fold {fold_no} on TESTING: {CNN1.metrics_names[0]} of {scores[0]}; {CNN1.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold_1.append(scores[1] * 100)
  loss_per_fold_1.append(scores[0])

  # Generate generalization metrics on TRAINING
  scores_train = CNN1.evaluate(X_train, y_train, verbose=0)
  print(f'Score for fold {fold_no} on TRAINING: {CNN1.metrics_names[0]} of {scores_train[0]}; {CNN1.metrics_names[1]} of {scores_train[1]*100}%')
  acc_per_fold_train_1.append(scores_train[1] * 100)
  loss_per_fold_train_1.append(scores_train[0])


  # Increase fold number
  fold_no = fold_no + 1


  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold on validation set')
for i in range(0, len(acc_per_fold_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_1[i]} - Accuracy: {acc_per_fold_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_1)} (+- {np.std(acc_per_fold_1)})')
print(f'> Loss: {np.mean(loss_per_fold_1)}')
print('------------------------------------------------------------------------')


print('------------------------------------------------------------------------')
print('Score per fold on training set')
for i in range(0, len(acc_per_fold_train_1)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold_train_1[i]} - Accuracy: {acc_per_fold_train_1[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_train_1)} (+- {np.std(acc_per_fold_train_1)})')
print(f'> Loss: {np.mean(loss_per_fold_train_1)}')
print('------------------------------------------------------------------------')


------------------------------------------------------------------------

Training set size 19292
Testing set size 4823

------------------------------------------------------------------------


------------------------------------------------------------------------

Predictions for fold 1 / with pretrained CNN1 / Complete dataset...
Score for fold 1 on TESTING: loss of 0.2146003395318985; accuracy of 90.8148467540741%
Score for fold 1 on TRAINING: loss of 0.2088318169116974; accuracy of 91.52498245239258%

------------------------------------------------------------------------

Training set size 19292
Testing set size 4823

------------------------------------------------------------------------


------------------------------------------------------------------------

Predictions for fold 2 / with pretrained CNN1 / Complete dataset...
Score for fold 2 on TESTING: loss of 0.19813622534275055; accuracy of 92.14181900024414%
Score for fold 2 on TRAINING: loss of 0.2129478007555008;

In [ ]:
modelcnn.load_weights('/content/drive/My Drive/MT/checkpoints/CNN1/CNNZhang25x25-8220')

In [ ]:
modelcnn.save('/content/drive/My Drive/MT/checkpoints/CNN1/CNN1_25')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/MT/checkpoints/CNN1/CNN1_25/assets


In [ ]:
january = np.load('/content/drive/My Drive/MT/2019/Jan2019_v2.npy')
january.shape

(1187, 710, 20)

In [ ]:
model = build_model1(input_shape = (25, 25, 20), batch_normalization=True, momentum = 0.99, activation = 'relu', loss='binary_crossentropy', lr = 0.001, optimizer= 'adam')

model.load_weights('/content/drive/My Drive/MT/checkpoints/CNNZhang25x25-8220')

In [ ]:
map = np.zeros((1163, 686))
i = 0
j = 0
patch_size = 25
margin = int(patch_size/2 - 0.5)

for i in range(january.shape[0] - patch_size + 1):
  for j in range(january.shape[1] - patch_size + 1):
    patch = january[i:i + patch_size, j:j + patch_size, :]
    patch = np.expand_dims(patch, axis=0)
    probability = model(patch, training = False)
    map[i,j] = probability

map1 = np.pad(map, 12, mode='constant')
print(map1.shape)

np.save('/content/drive/My Drive/MT/CNN25-Jan2019.npy', map1)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

cm_matrix = confusion_matrix(y_test, predict_class)
df_cm = pd.DataFrame(cm_matrix, ['0', '1'], ['0', '1'])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, cmap="YlGnBu", fmt = 'd')

tn, fp, fn, tp = cm_matrix.ravel()
print('True positives:', tn, "False positives:", fp, 'False negatives:', fn, 'True negatives:', tn)